In [ ]:
import json
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoModelForCausalLM, AutoTokenizer
from sklearn.metrics import accuracy_score, classification_report
from tqdm import tqdm
import os
import sys
import numpy as np

# ==========================================
# 1. 設定參數
# ==========================================
JSON_FILE = "/kaggle/input/knowledge/mmlu.json"
MODEL_NAME = "mistralai/Mistral-7B-v0.1"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 0.001

# ==========================================
# 2. 定義論文原版 Probe (LinearModel)
# ==========================================
class LinearModel(nn.Module):
    def __init__(self, input_dim):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(input_dim, input_dim)
        self.out = nn.Linear(input_dim, 2)
        self.activation = nn.ReLU()

    def forward(self, x):
        return self.out(self.activation(self.linear(x)))

# ==========================================
# 3. 輔助函數
# ==========================================
def format_json_prompt(item):
    prompt = f"Question: {item['question']}\n"
    for key in ['A', 'B', 'C', 'D']:
        choice_text = item['choices'].get(key, "")
        prompt += f"{key}. {choice_text}\n"
    prompt += "Answer:"
    return prompt

def compute_paper_metrics(correct_flags, abstain_flags):
    A = 0; B = 0; C = 0; D = 0
    for i in range(len(correct_flags)):
        if abstain_flags[i] == 1: 
            if correct_flags[i] == 1: B += 1
            else: D += 1
        else: 
            if correct_flags[i] == 1: A += 1
            else: C += 1
    
    total = A + B + C + D
    r_acc = A / (A + C) if (A + C) > 0 else 0.0
    er = (A - C) / total
    a_acc = (A + D) / total
    
    precision = D / (B + D) if (B + D) > 0 else 0.0
    recall = D / (C + D) if (C + D) > 0 else 0.0
    a_f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return r_acc, er, a_acc, a_f1, A, B, C, D

# ==========================================
# 4. 提取特徵
# ==========================================
def extract_features_baseline(data_list, model, tokenizer, desc="Processing"):
    features = []
    labels = []
    
    print(f"正在從 {desc} 提取特徵 (Paper Baseline Method)...")
    
    for item in tqdm(data_list):
        prompt = format_json_prompt(item)
        ground_truth = item['answer'].strip().upper()
        
        inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)
        
        with torch.no_grad():
            # 1. 提取 Hidden States
            outputs = model(**inputs, output_hidden_states=True)
            
            # === 關鍵差異 1: 只取最後一層 ===
            last_hidden_state = outputs.hidden_states[-1] # [1, seq_len, 4096]
            
            # === 關鍵差異 2: Mean Pooling (平均池化) ===
            mean_pooled = last_hidden_state.mean(dim=1) # [1, 4096]
            
            # 轉型
            feature_vector = mean_pooled.to(dtype=torch.float32)
            
            # 2. 生成答案
            generated_ids = model.generate(
                **inputs, 
                max_new_tokens=1, 
                do_sample=False, # 這裡讓他固定生成最高機率的token
                pad_token_id=tokenizer.pad_token_id
            )
            new_token_id = generated_ids[0][-1]
            pred_text = tokenizer.decode(new_token_id).strip().upper()
            
            # 3. 標註
            label = 1.0 if pred_text == ground_truth else 0.0
            
            features.append(feature_vector)
            labels.append(label)
            
    X = torch.cat(features, dim=0)
    y = torch.tensor(labels).long()
    return X, y

# ==========================================
# 5. 主程式
# ==========================================
def main():
    print(f"Using device: {DEVICE}")
    
    if not os.path.exists(JSON_FILE): return
    with open(JSON_FILE, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    
    # 載入模型
    print(f"Loading LLM: {MODEL_NAME}...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    if tokenizer.pad_token is None: tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, torch_dtype=torch.float16, device_map="auto")
    model.eval()
    
    # 提取特徵 (Baseline 方式)
    X_train, y_train = extract_features_baseline(raw_data['dev'], model, tokenizer, desc="Dev (Train)")
    X_test, y_test = extract_features_baseline(raw_data['test'], model, tokenizer, desc="Test (Eval)")
    
    # 釋放顯存
    del model
    torch.cuda.empty_cache()
    
    # 訓練 Probe
    print("\n" + "="*30)
    print("開始訓練 Paper Baseline Probe...")
    
    train_dataset = TensorDataset(X_train.to(DEVICE), y_train.to(DEVICE))
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    
    input_dim = X_train.shape[1]
    probe = LinearModel(input_dim).to(DEVICE)
    optimizer = optim.Adam(probe.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss() # 用 CrossEntropy 因為是 2 個 logits
    
    for epoch in range(EPOCHS):
        probe.train()
        epoch_loss = 0
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            outputs = probe(batch_x)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
        if (epoch + 1) % 10 == 0:
             print(f"Epoch {epoch+1:02d} | Loss: {epoch_loss/len(train_loader):.4f}")

    # 評估
    print("\n" + "="*30)
    print("🏆 論文基準線重現結果 (Paper Reproduction)")
    print("="*30)
    
    probe.eval()
    with torch.no_grad():
        logits = probe(X_test.to(DEVICE))
        probs = torch.softmax(logits, dim=1).cpu().numpy()
        
    # 取出 "Known" (Label 1) 的機率
    prob_known = probs[:, 1]
    
    threshold = 0.5
    final_preds = (prob_known > threshold).astype(int)
    
    paper_correct_flags = y_test.numpy().astype(int)
    paper_abstain_flags = 1 - final_preds
    
    r_acc, er, a_acc, a_f1, A, B, C, D = compute_paper_metrics(paper_correct_flags, paper_abstain_flags)
    
    print(f"總樣本數: {len(y_test)}")
    print(f"Counts: A={A}, B={B}, C={C}, D={D}")
    print("-" * 30)
    print(f"1. Reliable Accuracy (R-Acc) : {r_acc:.4f}")
    print(f"2. Effective Reliability (ER): {er:.4f}")
    print(f"3. Abstain Accuracy (A-Acc)  : {a_acc:.4f}")
    print(f"4. Abstain F1 Score (A-F1)   : {a_f1:.4f}")
    print("-" * 30)
    
    print(classification_report(paper_correct_flags, final_preds, target_names=["Unknown (0)", "Known (1)"]))

if __name__ == "__main__":
    main()

Using device: cuda
Loading LLM: mistralai/Mistral-7B-v0.1...


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

正在從 Dev (Train) 提取特徵 (Paper Baseline Method)...


100%|██████████| 1000/1000 [03:50<00:00,  4.35it/s]


正在從 Test (Eval) 提取特徵 (Paper Baseline Method)...


100%|██████████| 1000/1000 [03:57<00:00,  4.22it/s]



開始訓練 Paper Baseline Probe...
Epoch 10 | Loss: 0.1257

🏆 論文基準線重現結果 (Paper Reproduction)
總樣本數: 1000
Counts: A=470, B=134, C=235, D=161
------------------------------
1. Reliable Accuracy (R-Acc) : 0.6667
2. Effective Reliability (ER): 0.2350
3. Abstain Accuracy (A-Acc)  : 0.6310
4. Abstain F1 Score (A-F1)   : 0.4660
------------------------------
              precision    recall  f1-score   support

 Unknown (0)       0.55      0.41      0.47       396
   Known (1)       0.67      0.78      0.72       604

    accuracy                           0.63      1000
   macro avg       0.61      0.59      0.59      1000
weighted avg       0.62      0.63      0.62      1000

